# BentoML Demo: Titanic Survival Prediction with XGBoost


BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


This notebook demonstrates use BentoML to serve a model trained with the XGBoost framework, specifically using the Titanic Survival dataset.


Let's get started!
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=xgboost&ea=xgboost-tiantic-survival-prediction&dt=xgboost-tiantic-survival-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install -q --upgrade xgboost==0.90 numpy==1.18.5 pandas==1.0.4 bentoml

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import bentoml

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [3]:
!mkdir data
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   133k      0 --:--:-- --:--:-- --:--:--  133k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0  62273      0 --:--:-- --:--:-- --:--:-- 62273


In [4]:
train = pd.read_csv("./data/train.csv")
test  = pd.read_csv("./data/test.csv")
X_y_train = xgb.DMatrix(data=train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']], label= train['Survived'])
X_test    = xgb.DMatrix(data=test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])

In [5]:
train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Survived']].head()

,Pclass,Age,Fare,SibSp,Parch,Survived
0,3,22.0,7.2500,1,0,0
1,1,38.0,71.2833,1,0,1
2,3,26.0,7.9250,0,0,1
3,1,35.0,53.1000,1,0,1
4,3,35.0,8.0500,0,0,0


# Model Training

In [6]:
params = {
          'base_score': np.mean(train['Survived']),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

[15:41:43] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[15:41:43] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


In [7]:
y_test =  model.predict(X_test)
test['pred'] = y_test
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.341580
11,1,46.0,26.0000,0,0,0.413966


## Create BentoService for model serving

In [8]:
%%writefile xgboost_titanic_bento_service.py

import xgboost as xgb

import bentoml
from bentoml.frameworks.xgboost import XgboostModelArtifact
from bentoml.adapters import DataframeInput

@bentoml.env(infer_pip_packages=True)
@bentoml.artifacts([XgboostModelArtifact('model')])
class TitanicSurvivalPredictionXgBoost(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput(), batch=True)
    def predict(self, df):
        data = xgb.DMatrix(data=df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])
        return self.artifacts.model.predict(data)

Overwriting xgboost_titanic_bento_service.py


# Create BentoService saved bundle

In [9]:
# 1) import the custom BentoService defined above
from xgboost_titanic_bento_service import TitanicSurvivalPredictionXgBoost

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionXgBoost()
bento_service.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2020-09-22 15:42:22,025] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 15:42:22,375] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 15:42:23,380] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py
set BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py to '0.9.0.pre+3.gcebf2015'
[2020-09-22 15:42:27,403] INFO - BentoService bundle 'TitanicSurvivalPredictionXgBoost:20200922154223_4ACDA4' saved to: /Users/bozhaoyu/bentoml/repository/TitanicSurvivalPredictionXgBoost/20200922154223_4ACDA4


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [10]:
!bentoml serve TitanicSurvivalPredictionXgBoost:latest

[2020-09-22 15:43:00,146] INFO - Getting latest version TitanicSurvivalPredictionXgBoost:20200922154223_4ACDA4
[2020-09-22 15:43:00,146] INFO - Starting BentoML API server in development mode..
[2020-09-22 15:43:00,389] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 15:43:00,404] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 15:43:01,341] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[15:43:01] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squ

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve TitanicSurvivalPredictionXgBoost:latest --run-with-ngrok

Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.


### Send prediction requeset to the REST API server

Run the following `curl` command to send request data to REST API server and get a prediction result:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
localhost:5000/predict
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [11]:
!bentoml containerize TitanicSurvivalPredictionXgBoost:latest

[2020-09-22 15:43:25,886] INFO - Getting latest version TitanicSurvivalPredictionXgBoost:20200922154223_4ACDA4
Found Bento: /Users/bozhaoyu/bentoml/repository/TitanicSurvivalPredictionXgBoost/20200922154223_4ACDA4
[2020-09-22 15:43:25,932] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 15:43:25,958] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
Tag not specified, using tag parsed from BentoService: 'titanicsurvivalpredictionxgboost:20200922154223_4ACDA4'
Building Docker image titanicsurvivalpredictionxgboost:20200922154223_4ACDA4 from TitanicSurvivalPredictionXgBoost:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/ben

|Requirement already satisfied: aiohttp in /opt/conda/lib/python3.7/site-packages (from bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (3.6.2)
/Collecting pytz>=2011k
-Collecting scipy
/  Downloading scipy-1.5.2-cp37-cp37m-manylinux1_x86_64.whl (25.9 MB)
/Requirement already satisfied: Werkzeug>=0.15 in /opt/conda/lib/python3.7/site-packages (from flask->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (1.0.1)
|Requirement already satisfied: s3transfer<0.4.0,>=0.3.0 in /opt/conda/lib/python3.7/site-packages (from boto3->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (0.3.3)
\Installing collected packages: pytz, pandas, scipy, xgboost
/Successfully installed pandas-0.24.2 pytz-2020.1 scipy-1.5.2 xgboost-1.2.0
\ ---> aa4db686ba89
Step 8/15 : COPY . /bento
| ---> 4bafca014215
Step 9/15 : RUN if [ -d /bento/bundled_pip_dependencies ]; then pip install -U bundled_pip_dependencies/* ;fi
 ---> Running in 6d5fa028c021
-Processing ./bundled_pip_dependencies/BentoML-0.9.0

\Requirement already satisfied, skipping upgrade: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /opt/conda/lib/python3.7/site-packages (from requests->BentoML==0.9.0rc0+3.gcebf2015) (1.25.10)
Requirement already satisfied, skipping upgrade: botocore<1.19.0,>=1.18.2 in /opt/conda/lib/python3.7/site-packages (from boto3->BentoML==0.9.0rc0+3.gcebf2015) (1.18.2)
-Requirement already satisfied, skipping upgrade: jmespath<1.0.0,>=0.7.1 in /opt/conda/lib/python3.7/site-packages (from boto3->BentoML==0.9.0rc0+3.gcebf2015) (0.10.0)
/  Building wheel for BentoML (PEP 517): finished with status 'done'
  Created wheel for BentoML: filename=BentoML-0.9.0rc0+3.gcebf2015-py3-none-any.whl size=3064091 sha256=a49ace9338ca06346a0ec6749888d021774ab66106b4617a2935d82006d25f23
  Stored in directory: /root/.cache/pip/wheels/a0/45/41/62152db705af4ff47e7a3d6abf6247986eef4aa1b94a58d3b9
Successfully built BentoML
/Installing collected packages: BentoML
  Attempting uninstall: BentoML
    Found existing install

Next, you can docker push the image to your choice of registry for deployment, or run it locally for development and testing:

In [12]:
!docker run -p 5000:5000 titanicsurvivalpredictionxgboost:20200922154223_4ACDA4

[2020-09-22 22:47:14,344] INFO - Starting BentoML API server in production mode..
[2020-09-22 22:47:14,742] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-09-22 22:47:14 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-09-22 22:47:14 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-09-22 22:47:14 +0000] [1] [INFO] Using worker: sync
[2020-09-22 22:47:14 +0000] [11] [INFO] Booting worker with pid: 11
[2020-09-22 22:47:14 +0000] [12] [INFO] Booting worker with pid: 12
[2020-09-22 22:47:14 +0000] [13] [INFO] Booting worker with pid: 13
[2020-09-22 22:47:15,001] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-09-22 22:47:15,023] WARNING - Saved BentoService bundle version mismatch

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [13]:
import bentoml

loaded_svc = bentoml.load(saved_path)

result = loaded_svc.predict(test)
test['pred'] = result
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']]

[2020-09-22 15:47:22,359] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 15:47:22,360] WARNING - Module `xgboost_titanic_bento_service` already loaded, using existing imported module.
[15:47:22] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[2020-09-22 15:47:22,363] WARNING - pip package requirement pandas already exist
[2020-09-22 15:47:22,365] WARNING - pip package requirement xgboost already exist


,Pclass,Age,Fare,SibSp,Parch,pred
0,3,34.5,7.8292,0,0,0.341580
1,3,47.0,7.0000,1,0,0.341580
2,2,62.0,9.6875,0,0,0.371730
3,3,27.0,8.6625,0,0,0.341580
4,3,22.0,12.2875,1,1,0.341580
5,3,14.0,9.2250,0,0,0.341580
6,3,30.0,7.6292,0,0,0.341580
7,2,26.0,29.0000,1,1,0.472451
8,3,18.0,7.2292,0,0,0.341580
9,3,21.0,24.1500,2,0,0.341580


## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [14]:
!bentoml run TitanicSurvivalPredictionXgBoost:latest predict \
    --input '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' 

[2020-09-22 15:47:27,495] INFO - Getting latest version TitanicSurvivalPredictionXgBoost:20200922154223_4ACDA4
[2020-09-22 15:47:27,533] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 15:47:27,549] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 15:47:28,744] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[15:47:28] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[2020-09-22 15:47:31,407] INFO - {'service_name': 'TitanicSurvivalPredic

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

